In [ ]:
{
    "cells": [
        {
            "cell_type": "code",
            "execution_count": 1,
            "metadata": {
                "execution": {
                    "iopub.execute_input": "2021-12-09T15:42:30.890422Z",
                    "iopub.status.busy": "2021-12-09T15:42:30.890036Z",
                    "iopub.status.idle": "2021-12-09T15:42:37.774794Z",
                    "shell.execute_reply": "2021-12-09T15:42:37.774062Z",
                    "shell.execute_reply.started": "2021-12-09T15:42:30.890331Z"
                },
                "trusted": true
            },
            "outputs": [
                {
                    "name": "stderr",
                    "output_type": "stream",
                    "text": [
                        "2022-12-02 20:56:28.403071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA\n",
                        "To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.\n",
                        "2022-12-02 20:56:30.401887: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:\n",
                        "2022-12-02 20:56:30.402185: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:\n",
                        "2022-12-02 20:56:30.402198: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.\n"
                    ]
                }
            ],
            "source": [
                "from keras.preprocessing.text import Tokenizer\n",
                "from keras.utils import pad_sequences\n",
                "from nltk.tokenize import word_tokenize\n",
                "from nltk.stem import WordNetLemmatizer\n",
                "from nltk.corpus import stopwords\n",
                "from tensorflow.python.keras.layers import *\n",
                "from tensorflow.python.keras.models import Model\n",
                "import numpy as np \n",
                "import pandas as pd \n",
                "import re\n",
                "import nltk\n",
                "from preprocess import *\n",
                "from models import *"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 2,
            "metadata": {
                "execution": {
                    "iopub.execute_input": "2021-12-09T15:42:52.454527Z",
                    "iopub.status.busy": "2021-12-09T15:42:52.453714Z",
                    "iopub.status.idle": "2021-12-09T15:43:01.326923Z",
                    "shell.execute_reply": "2021-12-09T15:43:01.326227Z",
                    "shell.execute_reply.started": "2021-12-09T15:42:52.454487Z"
                },
                "trusted": true
            },
            "outputs": [
                {
                    "data": {
                        "text/html": [
                            "<div>\n",
                            "<style scoped>\n",
                            "    .dataframe tbody tr th:only-of-type {\n",
                            "        vertical-align: middle;\n",
                            "    }\n",
                            "\n",
                            "    .dataframe tbody tr th {\n",
                            "        vertical-align: top;\n",
                            "    }\n",
                            "\n",
                            "    .dataframe thead th {\n",
                            "        text-align: right;\n",
                            "    }\n",
                            "</style>\n",
                            "<table border=\"1\" class=\"dataframe\">\n",
                            "  <thead>\n",
                            "    <tr style=\"text-align: right;\">\n",
                            "      <th></th>\n",
                            "      <th>id</th>\n",
                            "      <th>qid1</th>\n",
                            "      <th>qid2</th>\n",
                            "      <th>question1</th>\n",
                            "      <th>question2</th>\n",
                            "      <th>is_duplicate</th>\n",
                            "    </tr>\n",
                            "  </thead>\n",
                            "  <tbody>\n",
                            "    <tr>\n",
                            "      <th>0</th>\n",
                            "      <td>0</td>\n",
                            "      <td>1</td>\n",
                            "      <td>2</td>\n",
                            "      <td>What is the step by step guide to invest in sh...</td>\n",
                            "      <td>What is the step by step guide to invest in sh...</td>\n",
                            "      <td>0</td>\n",
                            "    </tr>\n",
                            "    <tr>\n",
                            "      <th>1</th>\n",
                            "      <td>1</td>\n",
                            "      <td>3</td>\n",
                            "      <td>4</td>\n",
                            "      <td>What is the story of Kohinoor (Koh-i-Noor) Dia...</td>\n",
                            "      <td>What would happen if the Indian government sto...</td>\n",
                            "      <td>0</td>\n",
                            "    </tr>\n",
                            "    <tr>\n",
                            "      <th>2</th>\n",
                            "      <td>2</td>\n",
                            "      <td>5</td>\n",
                            "      <td>6</td>\n",
                            "      <td>How can I increase the speed of my internet co...</td>\n",
                            "      <td>How can Internet speed be increased by hacking...</td>\n",
                            "      <td>0</td>\n",
                            "    </tr>\n",
                            "    <tr>\n",
                            "      <th>3</th>\n",
                            "      <td>3</td>\n",
                            "      <td>7</td>\n",
                            "      <td>8</td>\n",
                            "      <td>Why am I mentally very lonely? How can I solve...</td>\n",
                            "      <td>Find the remainder when [math]23^{24}[/math] i...</td>\n",
                            "      <td>0</td>\n",
                            "    </tr>\n",
                            "    <tr>\n",
                            "      <th>4</th>\n",
                            "      <td>4</td>\n",
                            "      <td>9</td>\n",
                            "      <td>10</td>\n",
                            "      <td>Which one dissolve in water quikly sugar, salt...</td>\n",
                            "      <td>Which fish would survive in salt water?</td>\n",
                            "      <td>0</td>\n",
                            "    </tr>\n",
                            "  </tbody>\n",
                            "</table>\n",
                            "</div>"
                        ],
                        "text/plain": [
                            "   id  qid1  qid2                                          question1  \\\n",
                            "0   0     1     2  What is the step by step guide to invest in sh...   \n",
                            "1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   \n",
                            "2   2     5     6  How can I increase the speed of my internet co...   \n",
                            "3   3     7     8  Why am I mentally very lonely? How can I solve...   \n",
                            "4   4     9    10  Which one dissolve in water quikly sugar, salt...   \n",
                            "\n",
                            "                                           question2  is_duplicate  \n",
                            "0  What is the step by step guide to invest in sh...             0  \n",
                            "1  What would happen if the Indian government sto...             0  \n",
                            "2  How can Internet speed be increased by hacking...             0  \n",
                            "3  Find the remainder when [math]23^{24}[/math] i...             0  \n",
                            "4            Which fish would survive in salt water?             0  "
                        ]
                    },
                    "execution_count": 2,
                    "metadata": {},
                    "output_type": "execute_result"
                }
            ],
            "source": [
                "df = pd.read_csv(\"questions.csv\")\n",
                "df.head()"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 3,
            "metadata": {},
            "outputs": [
                {
                    "data": {
                        "text/plain": [
                            "(404351, 6)"
                        ]
                    },
                    "execution_count": 3,
                    "metadata": {},
                    "output_type": "execute_result"
                }
            ],
            "source": [
                "df.shape"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 4,
            "metadata": {},
            "outputs": [
                {
                    "data": {
                        "text/plain": [
                            "(100000, 6)"
                        ]
                    },
                    "execution_count": 4,
                    "metadata": {},
                    "output_type": "execute_result"
                }
            ],
            "source": [
                "#pick a sample of 5000 distinct random rows\n",
                "df = df.sample(n=100000, random_state=1)\n",
                "df.shape"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 5,
            "metadata": {
                "execution": {
                    "iopub.execute_input": "2021-12-07T01:21:29.482516Z",
                    "iopub.status.busy": "2021-12-07T01:21:29.482071Z",
                    "iopub.status.idle": "2021-12-07T01:21:29.56485Z",
                    "shell.execute_reply": "2021-12-07T01:21:29.564017Z",
                    "shell.execute_reply.started": "2021-12-07T01:21:29.482481Z"
                },
                "trusted": true
            },
            "outputs": [],
            "source": [
                "# question_1, question_2 = df['question1'].to_list(), df['question2'].to_list()\n",
                "# is_duplicate = df['is_duplicate'].to_list()\n",
                "# preprocess_neural(question_1, question_2, is_duplicate)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 6,
            "metadata": {
                "trusted": true
            },
            "outputs": [],
            "source": [
                "q1_inp, q2_inp, is_duplicate = df['question1'].to_list(), df['question2'].to_list(), df['is_duplicate'].to_list()"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "Acquired Test data"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 7,
            "metadata": {
                "trusted": true
            },
            "outputs": [],
            "source": [
                "MAX_WORDS_VOCAB = 200000\n",
                "tokenizer = Tokenizer(num_words = MAX_WORDS_VOCAB, lower=False, split=\" \")\n",
                "tokenizer.fit_on_texts(list(df['question1'].values.astype(str))+list(df['question2'].values.astype(str)))"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 8,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Number of words in vocabulary:  60949\n"
                    ]
                }
            ],
            "source": [
                "print(\"Number of words in vocabulary: \", len(tokenizer.word_index))"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 9,
            "metadata": {
                "trusted": true
            },
            "outputs": [],
            "source": [
                "\n",
                "q1_sequence = tokenizer.texts_to_sequences(df['question1'].values.astype(str))\n",
                "q1_sequence = pad_sequences(q1_sequence, maxlen = 128)\n",
                "\n",
                "q2_sequence = tokenizer.texts_to_sequences(df['question2'].values.astype(str))\n",
                "q2_sequence = pad_sequences(q2_sequence, maxlen = 128)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 10,
            "metadata": {},
            "outputs": [],
            "source": [
                "windex = tokenizer.word_index"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 11,
            "metadata": {
                "trusted": true
            },
            "outputs": [],
            "source": [
                "embedding_index = {}\n",
                "with open('glove.6B.300d.txt','r') as f:\n",
                "    for line in f:\n",
                "        values = line.split()\n",
                "        word = values[0]\n",
                "        vectors = np.asarray(values[1:], 'float32')\n",
                "        embedding_index[word] = vectors\n",
                "    f.close()"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 12,
            "metadata": {
                "trusted": true
            },
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "(60950, 300)\n"
                    ]
                }
            ],
            "source": [
                "embedding_matrix = np.random.random((len(windex)+1, 300))\n",
                "\n",
                "for word, i in windex.items():\n",
                "    embedding_vector = embedding_index.get(word)\n",
                "    if embedding_vector is not None:\n",
                "        embedding_matrix[i] = embedding_vector\n",
                "\n",
                "print(embedding_matrix.shape)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 13,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "60950\n"
                    ]
                }
            ],
            "source": [
                "print(len(windex)+1)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 14,
            "metadata": {},
            "outputs": [],
            "source": [
                "#split the data into 70-20-10 train-validation-test with random state 42\n",
                "from sklearn.model_selection import train_test_split\n",
                "q1_train, q1_test, q2_train, q2_test, y_train, y_test = train_test_split(q1_sequence, q2_sequence, is_duplicate, test_size=0.1, random_state=42)\n",
                "q1_train, q1_val, q2_train, q2_val, y_train, y_val = train_test_split(q1_train, q2_train, y_train, test_size=0.2, random_state=42)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 15,
            "metadata": {},
            "outputs": [],
            "source": [
                "import tensorflow as tf\n",
                "y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)\n",
                "y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)\n",
                "y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 16,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "(100000,)\n"
                    ]
                }
            ],
            "source": [
                "is_duplicate = np.array(is_duplicate)\n",
                "print(is_duplicate.shape)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 17,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Train:  [0.63294444 0.36705556]\n",
                        "Validation:  [0.6358333  0.36416668]\n",
                        "Test:  [0.6291 0.3709]\n"
                    ]
                }
            ],
            "source": [
                "#print the ratio of positive and negative samples in train, validation and test\n",
                "y_train, y_val, y_test = np.array(y_train), np.array(y_val), np.array(y_test)\n",
                "print(\"Train: \", sum(y_train)/len(y_train))\n",
                "print(\"Validation: \", sum(y_val)/len(y_val))\n",
                "print(\"Test: \", sum(y_test)/len(y_test))"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 107,
            "metadata": {},
            "outputs": [],
            "source": [
                "# %load models.py\n",
                "from keras.layers import *\n",
                "from keras.models import Model\n",
                "import tensorflow as tf\n",
                "import numpy as np \n",
                "from keras import Sequential\n",
                "from keras.optimizers import adam_v2\n",
                "import keras as K\n",
                "\n",
                "class NeuralModels:\n",
                "    def __init__(self, emb_mat, vocab_size = -1, loss = \"binary_crossentropy\", epochs = 10, optimizer = \"adam\", metrics = [\"accuracy\"]):\n",
                "        self.epochs = epochs\n",
                "        self.optimizer = optimizer\n",
                "        self.loss = loss\n",
                "        self.metrics = metrics\n",
                "        self.vocab_size = vocab_size\n",
                "        self.embedding_matrix = emb_mat\n",
                "        self.model = Sequential()\n",
                "\n",
                "class CBOW(NeuralModels):\n",
                "    def __init__(self, emb_mat, vocab_size = -1, loss = \"binary_crossentropy\", epochs = 10, optimizer = \"adam\", metrics = [\"accuracy\"]):\n",
                "        super().__init__(emb_mat, vocab_size, loss, epochs, optimizer, metrics)\n",
                "    \n",
                "    def fit(self,xtrain, xval, ytrain, yval):\n",
                "        self.model.add(Dense(300, input_shape = (900,), activation = \"sigmoid\"))\n",
                "        self.model.add(Dropout(0.1))\n",
                "        self.model.add(Dense(200, activation = \"sigmoid\"))\n",
                "        self.model.add(Dropout(0.1))\n",
                "        self.model.add(Dense(100, activation = \"sigmoid\"))\n",
                "        self.model.add(Dropout(0.1))\n",
                "        self.model.add(Dense(2, activation = \"softmax\"))\n",
                "        self.model.compile(loss = self.loss, optimizer = self.optimizer, metrics = self.metrics)\n",
                "        self.model.fit(xtrain, ytrain, epochs = self.epochs, validation_data = (xval, yval), batch_size = 64, verbose = 1)\n",
                "\n",
                "    def get_model_summary(self):\n",
                "        self.model.summary()\n",
                "    \n",
                "    def predict(self, xtest):\n",
                "        return self.model.predict(xtest)\n",
                "\n",
                "class LsTM(NeuralModels):\n",
                "    def __init__(self, emb_mat, vocab_size = -1, loss = \"binary_crossentropy\", epochs = 10, optimizer = \"adam\", metrics = [\"accuracy\"]):\n",
                "        super().__init__(emb_mat, vocab_size, loss, epochs, optimizer, metrics)\n",
                "    \n",
                "    def train_model(self):\n",
                "        inp1 = Input(shape = (128,))\n",
                "        inp2 = Input(shape = (128,))\n",
                "        emb1 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp1)\n",
                "        emb2 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp2)\n",
                "        concat = Concatenate(axis = -1)([emb1 + emb2, emb1 - emb2, emb1 * emb2])\n",
                "        lstm = LSTM(150, return_sequences=False, dropout=0.2, return_state=True)(concat)\n",
                "        out = Dense(2, activation = \"softmax\")(lstm[2])\n",
                "        self.model = Model(inputs = [inp1, inp2], outputs = out)\n",
                "        self.model.compile(loss = self.loss, optimizer = adam_v2.Adam(learning_rate=0.0008), metrics = self.metrics)\n",
                "\n",
                "    def get_model_summary(self):\n",
                "        self.model.summary()\n",
                "    \n",
                "    def predict(self, xtest):\n",
                "        return self.model.predict(xtest)\n",
                "\n",
                "class BiLSTM(NeuralModels):\n",
                "    def __init__(self, emb_mat, vocab_size = -1, loss = \"binary_crossentropy\", epochs = 10, optimizer = \"adam\", metrics = [\"accuracy\"]):\n",
                "        super().__init__(emb_mat, vocab_size, loss, epochs, optimizer, metrics)\n",
                "    \n",
                "    def train_model(self):\n",
                "        inp1 = Input(shape = (128,))\n",
                "        inp2 = Input(shape = (128,))\n",
                "        emb1 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp1)\n",
                "        emb2 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp2)\n",
                "        concat = Concatenate(axis = -1)([emb1 + emb2, emb1 - emb2, emb1 * emb2])\n",
                "        out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, kernel_regularizer='l2', dropout=0.1, return_sequences=True))(concat)\n",
                "        out = tf.keras.backend.mean(out, axis=1, keepdims=False)\n",
                "        output = tf.keras.layers.Dense(2, kernel_regularizer='l2', activation='softmax')(out)\n",
                "        self.model = Model(inputs = [inp1, inp2], outputs = output)\n",
                "        self.model.compile(loss = self.loss, optimizer = self.optimizer, metrics = self.metrics)\n",
                "\n",
                "    def get_model_summary(self):\n",
                "        self.model.summary()\n",
                "    \n",
                "    def predict(self, xtest):\n",
                "        return self.model.predict(xtest)\n",
                "\n",
                "class LsTM_Attention(NeuralModels):\n",
                "    def __init__(self, emb_mat, vocab_size = -1, loss = \"binary_crossentropy\", epochs = 10, optimizer = \"adam\", metrics = [\"accuracy\"]):\n",
                "        super().__init__(emb_mat, vocab_size, loss, epochs, optimizer, metrics)\n",
                "    \n",
                "    def train_model(self):\n",
                "        inp1 = Input(shape = (128,))\n",
                "        inp2 = Input(shape = (128,))\n",
                "        emb1 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp1)\n",
                "        emb2 = Embedding(output_dim=300, weights = [self.embedding_matrix], trainable = False, input_dim=self.vocab_size, input_length=128)(inp2)\n",
                "        \n",
                "        lstm1 = LSTM(150, return_sequences=True, dropout=0.1, return_state=True)(emb1)\n",
                "        lstm2 = LSTM(150, return_sequences=True, dropout=0.1, return_state=True)(emb2)\n",
                "\n",
                "        attention = dot([lstm1[0], lstm2[0]], axes=[2, 2])\n",
                "        u_norm = Softmax(axis=-1)(attention)\n",
                "        v_norm = Softmax(axis=1)(attention)\n",
                "\n",
                "\n",
                "        u = dot([u_norm, lstm1[0]], axes=[1, 1])\n",
                "        v = dot([v_norm, lstm2[0]], axes=[1, 1])\n",
                "\n",
                "        WU_bar = Dense(150)(u[:, -1, :])\n",
                "        WV_bar = Dense(150)(v[:, -1, :])\n",
                "        VU = Dense(150)(lstm1[0][:, -1, :])\n",
                "        VV = Dense(150)(lstm2[0][:, -1, :])\n",
                "\n",
                "        ufinal = Add()([WU_bar, VU])\n",
                "        vfinal = Add()([WV_bar, VV])\n",
                "\n",
                "        ufinal = Activation('tanh')(ufinal)\n",
                "        vfinal = Activation('tanh')(vfinal)\n",
                "        \n",
                "        concat = Concatenate(axis = -1)([ufinal, vfinal])\n",
                "        out = Dense(2, activation = \"softmax\")(concat)\n",
                "        self.model = Model(inputs = [inp1, inp2], outputs = out)\n",
                "        self.model.compile(loss = self.loss, optimizer = self.optimizer, metrics = self.metrics)\n",
                "\n",
                "\n",
                "    def get_model_summary(self):\n",
                "        self.model.summary()\n",
                "    \n",
                "    def predict(self, xtest):\n",
                "        return self.model.predict(xtest)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 108,
            "metadata": {},
            "outputs": [],
            "source": [
                "model = LsTM_Attention(embedding_matrix, len(windex) + 1, loss=\"categorical_crossentropy\")"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 109,
            "metadata": {},
            "outputs": [],
            "source": [
                "model.train_model()"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 110,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Model: \"model_6\"\n",
                        "__________________________________________________________________________________________________\n",
                        " Layer (type)                   Output Shape         Param #     Connected to                     \n",
                        "==================================================================================================\n",
                        " input_45 (InputLayer)          [(None, 128)]        0           []                               \n",
                        "                                                                                                  \n",
                        " input_46 (InputLayer)          [(None, 128)]        0           []                               \n",
                        "                                                                                                  \n",
                        " embedding_44 (Embedding)       (None, 128, 300)     18285000    ['input_45[0][0]']               \n",
                        "                                                                                                  \n",
                        " embedding_45 (Embedding)       (None, 128, 300)     18285000    ['input_46[0][0]']               \n",
                        "                                                                                                  \n",
                        " lstm_40 (LSTM)                 [(None, 128, 150),   270600      ['embedding_44[0][0]']           \n",
                        "                                 (None, 150),                                                     \n",
                        "                                 (None, 150)]                                                     \n",
                        "                                                                                                  \n",
                        " lstm_41 (LSTM)                 [(None, 128, 150),   270600      ['embedding_45[0][0]']           \n",
                        "                                 (None, 150),                                                     \n",
                        "                                 (None, 150)]                                                     \n",
                        "                                                                                                  \n",
                        " dot_24 (Dot)                   (None, 128, 128)     0           ['lstm_40[0][0]',                \n",
                        "                                                                  'lstm_41[0][0]']                \n",
                        "                                                                                                  \n",
                        " softmax_4 (Softmax)            (None, 128, 128)     0           ['dot_24[0][0]']                 \n",
                        "                                                                                                  \n",
                        " softmax_5 (Softmax)            (None, 128, 128)     0           ['dot_24[0][0]']                 \n",
                        "                                                                                                  \n",
                        " dot_25 (Dot)                   (None, 128, 150)     0           ['softmax_4[0][0]',              \n",
                        "                                                                  'lstm_40[0][0]']                \n",
                        "                                                                                                  \n",
                        " dot_26 (Dot)                   (None, 128, 150)     0           ['softmax_5[0][0]',              \n",
                        "                                                                  'lstm_41[0][0]']                \n",
                        "                                                                                                  \n",
                        " tf.__operators__.getitem_281 (  (None, 150)         0           ['dot_25[0][0]']                 \n",
                        " SlicingOpLambda)                                                                                 \n",
                        "                                                                                                  \n",
                        " tf.__operators__.getitem_283 (  (None, 150)         0           ['lstm_40[0][0]']                \n",
                        " SlicingOpLambda)                                                                                 \n",
                        "                                                                                                  \n",
                        " tf.__operators__.getitem_282 (  (None, 150)         0           ['dot_26[0][0]']                 \n",
                        " SlicingOpLambda)                                                                                 \n",
                        "                                                                                                  \n",
                        " tf.__operators__.getitem_284 (  (None, 150)         0           ['lstm_41[0][0]']                \n",
                        " SlicingOpLambda)                                                                                 \n",
                        "                                                                                                  \n",
                        " dense_11 (Dense)               (None, 150)          22650       ['tf.__operators__.getitem_281[0]\n",
                        "                                                                 [0]']                            \n",
                        "                                                                                                  \n",
                        " dense_13 (Dense)               (None, 150)          22650       ['tf.__operators__.getitem_283[0]\n",
                        "                                                                 [0]']                            \n",
                        "                                                                                                  \n",
                        " dense_12 (Dense)               (None, 150)          22650       ['tf.__operators__.getitem_282[0]\n",
                        "                                                                 [0]']                            \n",
                        "                                                                                                  \n",
                        " dense_14 (Dense)               (None, 150)          22650       ['tf.__operators__.getitem_284[0]\n",
                        "                                                                 [0]']                            \n",
                        "                                                                                                  \n",
                        " add_2 (Add)                    (None, 150)          0           ['dense_11[0][0]',               \n",
                        "                                                                  'dense_13[0][0]']               \n",
                        "                                                                                                  \n",
                        " add_3 (Add)                    (None, 150)          0           ['dense_12[0][0]',               \n",
                        "                                                                  'dense_14[0][0]']               \n",
                        "                                                                                                  \n",
                        " activation_517 (Activation)    (None, 150)          0           ['add_2[0][0]']                  \n",
                        "                                                                                                  \n",
                        " activation_518 (Activation)    (None, 150)          0           ['add_3[0][0]']                  \n",
                        "                                                                                                  \n",
                        " concatenate_9 (Concatenate)    (None, 300)          0           ['activation_517[0][0]',         \n",
                        "                                                                  'activation_518[0][0]']         \n",
                        "                                                                                                  \n",
                        " dense_15 (Dense)               (None, 2)            602         ['concatenate_9[0][0]']          \n",
                        "                                                                                                  \n",
                        "==================================================================================================\n",
                        "Total params: 37,202,402\n",
                        "Trainable params: 632,402\n",
                        "Non-trainable params: 36,570,000\n",
                        "__________________________________________________________________________________________________\n"
                    ]
                }
            ],
            "source": [
                "model.get_model_summary()"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 111,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "(18000, 128)\n"
                    ]
                }
            ],
            "source": [
                "print(q1_val.shape)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 112,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Epoch 1/4\n",
                        "9000/9000 [==============================] - 1253s 139ms/step - loss: 0.5809 - accuracy: 0.6902 - val_loss: 0.5423 - val_accuracy: 0.7200\n",
                        "Epoch 2/4\n",
                        "9000/9000 [==============================] - 1242s 138ms/step - loss: 0.5190 - accuracy: 0.7422 - val_loss: 0.5348 - val_accuracy: 0.7373\n",
                        "Epoch 3/4\n",
                        "9000/9000 [==============================] - 1238s 138ms/step - loss: 0.4668 - accuracy: 0.7751 - val_loss: 0.5124 - val_accuracy: 0.7573\n",
                        "Epoch 4/4\n",
                        "9000/9000 [==============================] - 1248s 139ms/step - loss: 0.4030 - accuracy: 0.8133 - val_loss: 0.4988 - val_accuracy: 0.7642\n"
                    ]
                },
                {
                    "data": {
                        "text/plain": [
                            "<keras.callbacks.History at 0x7f30642141c0>"
                        ]
                    },
                    "execution_count": 112,
                    "metadata": {},
                    "output_type": "execute_result"
                }
            ],
            "source": [
                "model.model.fit([q1_train, q2_train], y_train, epochs = 4, validation_data = ([q1_val, q2_val], y_val), batch_size = 8, validation_batch_size=4, verbose = 1)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 113,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "313/313 [==============================] - 31s 87ms/step\n",
                        "Accuracy:  0.7751\n",
                        "F1 Score:  0.7064308681672025\n"
                    ]
                }
            ],
            "source": [
                "y_pred = model.predict([q1_test, q2_test])\n",
                "y_pred1d, y_actual1d = [], []\n",
                "for i in range(len(y_test)):\n",
                "    if(y_test[i][0] == 1):\n",
                "        y_actual1d.append(0)\n",
                "    else:\n",
                "        y_actual1d.append(1)\n",
                "\n",
                "for i in range(len(y_pred)):\n",
                "    if(y_pred[i][0] > y_pred[i][1]):\n",
                "        y_pred1d.append(0)\n",
                "    else:\n",
                "        y_pred1d.append(1)\n",
                "\n",
                "from sklearn.metrics import accuracy_score, f1_score\n",
                "print(\"Accuracy: \", accuracy_score(y_actual1d, y_pred1d))\n",
                "print(\"F1 Score: \", f1_score(y_actual1d, y_pred1d))"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": []
        }
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3.10.4 64-bit",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "codemirror_mode": {
                "name": "ipython",
                "version": 3
            },
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.10.4"
        },
        "vscode": {
            "interpreter": {
                "hash": "916dbcbb3f70747c44a77c7bcd40155683ae19c65e1c03b4aa3499c5328201f1"
            }
        }
    },
    "nbformat": 4,
    "nbformat_minor": 4
}